In [9]:
import quimb as qu
import quimb.tensor as qtn
import numpy as np
import scipy
import scipy.sparse.linalg as sla
import pickle


In [10]:
with open('sca_xs.pkl', 'rb') as file:
    sca_xs = pickle.load(file)
with open('sca_zs.pkl', 'rb') as file:
    sca_zs = pickle.load(file)
with open('scd_xs.pkl', 'rb') as file:
    scd_xs = pickle.load(file)
with open('scd_zs.pkl', 'rb') as file:
    scd_zs = pickle.load(file)

scd_xs=[-scd_x for scd_x in scd_xs]
scd_zs=[-scd_z for scd_z in scd_zs]

In [11]:
rho=sca_xs[5].identity()
left_mpos=sca_xs+sca_zs+scd_xs+scd_zs+[scd_xs[0].identity()]*(2*len(scd_xs))
right_mpos=sca_xs+sca_zs+[scd_xs[0].identity()]*(2*len(scd_xs))+scd_xs+scd_zs


In [12]:

length=rho.L

rho=qtn.MatrixProductOperator(rho.arrays)
rho_conj=qtn.MatrixProductOperator([arr.conj() for arr in rho.arrays])
for i in range(len(left_mpos)):
    left_mpo=qtn.MatrixProductOperator(left_mpos[i].arrays)
    left_mpo=left_mpo.reindex_lower_sites('c{}')
    left_mpo=left_mpo.reindex_upper_sites('b{}')
    right_mpo=qtn.MatrixProductOperator(right_mpos[i].arrays)
    right_mpo=right_mpo.reindex_upper_sites('d{}')
    right_mpo=right_mpo.reindex_lower_sites('a{}')
    left_mpos[i]=left_mpo
    right_mpos[i]=right_mpo
    
rho=rho.reindex_upper_sites('c{}')
rho=rho.reindex_lower_sites('d{}')
rho_conj=rho_conj.reindex_lower_sites('a{}')
rho_conj=rho_conj.reindex_upper_sites('b{}')

In [13]:


'''  _____
    |a    |
--[rh_conj]   
    |b    |
--[lmpo]  |      
    |c    |
--[rho]   |
    |d    |
--[rmpo]  |
    |a____|
'''
#Get the right tensors
right_sides=[]
for j in range(len(left_mpos)):
    rs=[]
    rtensors=[rho[-1],rho_conj[-1],left_mpos[j][-1],right_mpos[j][-1]]
    rs.append(qtn.tensor_contract(*rtensors))
    for i in range(length-3):
        rtensors=[rho[-2-i],rho_conj[-2-i],left_mpos[j][-2-i],right_mpos[j][-2-i],rs[-1]]
        rs.append(qtn.tensor_contract(*rtensors))
    right_sides.append(rs)

    


In [14]:
#Defie the linear operator
left_inds=[*rho_conj[0].inds[-2:],*rho_conj[1].inds[-3:]]
right_inds=[*rho[0].inds[-2:],*rho[1].inds[-3:]]
vec0=qtn.tensor_contract(rho[0],rho[1]).transpose(*right_inds).data.reshape(-1)
def matvec(vec):
    vec_tens=qtn.Tensor(vec.reshape((*rho[0].shape[-2:],*rho[1].shape[-3:])),right_inds)
    result_vec=np.zeros(vec.shape,dtype=np.complex128)
    for i in range(len(left_mpos)):
        tens=[vec_tens,left_mpos[i][0],left_mpos[i][1],right_mpos[i][0],right_mpos[i][1],right_sides[i][-1]]
        result_vec+=qtn.tensor_contract(*tens).transpose(*left_inds).data.reshape(-1)
    return result_vec
linop=sla.LinearOperator(shape=(len(vec0),len(vec0)),matvec=matvec)


In [15]:
#Find the eigenvectors of the first part and put them in the MPO
_,new_vec=sla.eigsh(linop,which='LA',v0=vec0,tol=1e-7,maxiter=100)
new_tens=qtn.Tensor(new_vec.T[0].reshape((*rho[0].shape[-2:],*rho[1].shape[-3:])),right_inds)
split_opts={'method':'svd','cutoff':1e-7,'cutoff_mode':'rel','max_bond':100}
splited=new_tens.split(right_inds[:2],absorb="right",**split_opts,ltags=rho[0].tags,rtags=rho[1].tags)
rho[0]=splited.tensors[0]
rho[0]=rho[0].transpose(rho[0].inds[-1],*rho[0].inds[:-1])
rho[1]=splited.tensors[1]
rho_conj[0]=qtn.Tensor(rho[0].data.transpose(0,2,1).conj(),rho_conj[0].inds,tags=rho_conj[0].tags)
rho_conj[1]=qtn.Tensor(rho[1].data.transpose(0,1,3,2).conj(),rho_conj[1].inds,tags=rho_conj[1].tags)

In [16]:
#Get the First Left side
left_sides=[]
for j in range(len(left_mpos)):
    ltensors=[rho[0],rho_conj[0],left_mpos[j][0],right_mpos[j][0]]
    left_sides.append([qtn.tensor_contract(*ltensors)])

for i in range(1,length-2):
    print(i)
    left_inds=[rho_conj[i].inds[0],rho_conj[i].inds[2],rho_conj[i].inds[3],rho_conj[i+1].inds[1],rho_conj[i+1].inds[2],rho_conj[i+1].inds[3]]
    right_inds=[rho[i].inds[0],rho[i].inds[2],rho[i].inds[3],rho[i+1].inds[1],rho[i+1].inds[2],rho[i+1].inds[3]]
    vec0=qtn.tensor_contract(rho[i],rho[i+1]).transpose(*right_inds).data.reshape(-1)
    def matvec(vec):
        vec_tens=qtn.Tensor(vec.reshape((rho[i].shape[0],*rho[i].shape[-2:],*rho[i+1].shape[-3:])),right_inds)
        result_vec=np.zeros(vec.shape,dtype=np.complex128)
        for j in range(len(left_mpos)):
            tens=[vec_tens,left_mpos[j][i],left_mpos[j][i+1],right_mpos[j][i],right_mpos[j][i+1],left_sides[j][-1],right_sides[j][-i-1]]
            result_vec+=qtn.tensor_contract(*tens).transpose(*left_inds).data.reshape(-1)
        return result_vec
    linop=sla.LinearOperator(shape=(len(vec0),len(vec0)),matvec=matvec)
    _,new_vec=sla.eigsh(linop,k=1,which='SM',v0=vec0,tol=1e-7,maxiter=100)
    new_tens=qtn.Tensor(new_vec.reshape((rho[i].shape[0],*rho[i].shape[-2:],*rho[i+1].shape[-3:])),right_inds)
    split_opts={'method':'svd','cutoff':1e-7,'cutoff_mode':'rel','max_bond':100}
    splited=new_tens.split([right_inds[0]]+right_inds[:3],absorb="right",**split_opts,ltags=rho[i].tags,rtags=rho[i+1].tags)
    rho[i]=splited.tensors[0]
    rho[i]=rho[i].transpose(rho[i].inds[0],rho[i].inds[3],rho[i].inds[1],rho[i].inds[2])
    rho[i+1]=splited.tensors[1]
    rho_conj[i]=qtn.Tensor(rho[i].data.transpose(0,1,3,2).conj(),rho_conj[i].inds,tags=rho_conj[i].tags)
    rho_conj[i+1]=qtn.Tensor(rho[i+1].data.transpose(0,1,3,2).conj(),rho_conj[i+1].inds,tags=rho_conj[i+1].tags)
    for j in range(len(left_mpos)):
        ltensors=[rho[i],rho_conj[i],left_mpos[j][i],right_mpos[j][i],left_sides[j][-1]]
        left_sides[j].append(qtn.tensor_contract(*ltensors))



1
2
3
4
5
6
7
8
9


In [17]:
#Defie the linear operator
left_inds=[rho_conj[-2].inds[0],rho_conj[-2].inds[2],rho_conj[-2].inds[3],rho_conj[-1].inds[1],rho_conj[-1].inds[2]]
right_inds=[rho[-2].inds[0],rho[-2].inds[2],rho[-2].inds[3],rho[-1].inds[1],rho[-1].inds[2]]
vec0=qtn.tensor_contract(rho[-2],rho[-1]).transpose(*right_inds).data.reshape(-1)
tens_shape=(rho[-2].shape[0],rho[-2].shape[2],rho[-2].shape[3],rho[-1].shape[1],rho[-1].shape[2])
def matvec(vec):
    vec_tens=qtn.Tensor(vec.reshape(tens_shape),right_inds)
    result_vec=np.zeros(vec.shape,dtype=np.complex128)
    for i in range(len(left_mpos)):
        tens=[vec_tens,left_mpos[i][-2],left_mpos[i][-1],right_mpos[i][-2],right_mpos[i][-1],left_sides[i][-1]]
        result_vec+=qtn.tensor_contract(*tens).transpose(*left_inds).data.reshape(-1)
    return result_vec
linop=sla.LinearOperator(shape=(len(vec0),len(vec0)),matvec=matvec)


In [18]:
#Find the eigenvectors of the first part and put them in the MPO
_,new_vec=sla.eigsh(linop,which='LA',v0=vec0,tol=1e-7,maxiter=100)
new_tens=qtn.Tensor(new_vec.T[0].reshape(tens_shape),right_inds)
split_opts={'method':'svd','cutoff':1e-7,'cutoff_mode':'rel','max_bond':100}
splited=new_tens.split([right_inds[0]]+right_inds[:3],absorb="left",**split_opts,ltags=rho[-2].tags,rtags=rho[-1].tags)
rho[-2]=splited.tensors[0]
rho[-2]=rho[-2].transpose(rho[-2].inds[0],rho[-2].inds[3],rho[-2].inds[1],rho[-2].inds[2])
rho[-1]=splited.tensors[1]
rho_conj[-2]=qtn.Tensor(rho[-2].data.transpose(0,1,3,2).conj(),rho_conj[-2].inds,tags=rho_conj[-2].tags)
rho_conj[-1]=qtn.Tensor(rho[-1].data.transpose(0,2,1).conj(),rho_conj[-1].inds,tags=rho_conj[-1].tags)

In [19]:
#Get the First Left side
right_sides=[]
for j in range(len(left_mpos)):
    rtensors=[rho[-1],rho_conj[-1],left_mpos[j][-1],right_mpos[j][-1]]
    right_sides.append([qtn.tensor_contract(*rtensors)])

for i in range(2,length-2):
    print(i)
    left_inds=[rho_conj[-(i+1)].inds[0],rho_conj[-(i+1)].inds[2],rho_conj[-(i+1)].inds[3],rho_conj[-i].inds[1],rho_conj[-i].inds[2],rho_conj[-i].inds[3]]
    right_inds=[rho[-(i+1)].inds[0],rho[-(i+1)].inds[2],rho[-(i+1)].inds[3],rho[-i].inds[1],rho[-i].inds[2],rho[-i].inds[3]]
    vec0=qtn.tensor_contract(rho[-i],rho[-(i+1)]).transpose(*right_inds).data.reshape(-1)
    ten_shape=(rho[-(i+1)].shape[0],*rho[-(i+1)].shape[-2:],*rho[-i].shape[-3:])
    def matvec(vec):
        vec_tens=qtn.Tensor(vec.reshape(ten_shape),right_inds)
        result_vec=np.zeros(vec.shape,dtype=np.complex128)
        for j in range(len(left_mpos)):
            tens=[vec_tens,left_mpos[j][-(i+1)],left_mpos[j][-i],right_mpos[j][-(i+1)],right_mpos[j][-i],left_sides[j][-i],right_sides[j][-1]]
            result_vec+=qtn.tensor_contract(*tens).transpose(*left_inds).data.reshape(-1)
        return result_vec
    linop=sla.LinearOperator(shape=(len(vec0),len(vec0)),matvec=matvec)
    _,new_vec=sla.eigsh(linop,k=1,which='SM',v0=vec0,tol=1e-7,maxiter=100)
    new_tens=qtn.Tensor(new_vec.reshape(ten_shape),right_inds)
    split_opts={'method':'svd','cutoff':1e-7,'cutoff_mode':'abs','max_bond':100}
    splited=new_tens.split([right_inds[0]]+right_inds[:3],absorb="left",**split_opts,ltags=rho[-(i+1)].tags,rtags=rho[-i].tags)
    rho[-(i+1)]=splited.tensors[0]
    rho[-(i+1)]=rho[-(i+1)].transpose(rho[-(i+1)].inds[0],rho[-(i+1)].inds[3],rho[-(i+1)].inds[1],rho[-(i+1)].inds[2])
    rho[-i]=splited.tensors[1]
    rho_conj[-(i+1)]=qtn.Tensor(rho[-(i+1)].data.transpose(0,1,3,2).conj(),rho_conj[-(i+1)].inds,tags=rho_conj[-(i+1)].tags)
    rho_conj[-i]=qtn.Tensor(rho[-i].data.transpose(0,1,3,2).conj(),rho_conj[-i].inds,tags=rho_conj[-i].tags)
    for j in range(len(left_mpos)):
        rtensors=[rho[-i],rho_conj[-i],left_mpos[j][-i],right_mpos[j][-i],right_sides[j][-1]]
        right_sides[j].append(qtn.tensor_contract(*rtensors))


2
3
4
5
6
7
8
9
